## (Упрощенная) ERD сущностей в Clickup
![test](ClickupERD.png "Title")

 Команда - основная верхнеуровневая сущность. Название говорит само за себя. Нас интересует исключительно в разрезе team_id, который используется в некоторых эндпойнтах API
 В команде может быть несколько воркспейсов, в которых, в свою очередь, содержатся листы, которые могут быть сгруппированы в папки, но могут быть и прямыми детьми воркспейса.
 Лист - основная сущность, интересующая нас в контексте сборки датасетов, т.к. это коллекция тасков, которые, в свою очередь, содержат всю интересующую нас информацию (дата создания, статус и т.п.). Стоит обратить внимание, что мы можем определить кастомные поля для использования в тасках (например, причина отвала и т.п.). При запросе к API эти поля будут доступны по отдельному ключу custom_fields.
В интерфейсе все это выглядит как-то так:

![Interface](Entities.png "Ent")

In [1]:
import pandas as pd
import json
import ClickUpAPI as cua
import numpy as np
from ast import literal_eval
from math import ceil
from typing import List
from urllib.parse import quote_plus

Первым делом, как и для практически любого API, нам понадобится токен. В Кликапе есть два вида токенов: персональный, который можно получить в секции Apps настроек своего профиля и полноценный authorization code grant OAuth2. Здесь мы будем использовать персональный токен. ![SegmentLocal](PersToken3.gif "segment")

In [2]:
token = os.environ.get("ClickUpToken")

In [3]:
# Проверим, правильно ли API возвращает юзера, от лица которого делаются запросы:
test_class = cua.ClickupClient(token)
res = test_class.get_user()
print(res)

{'id': 8867829, 'username': 'Bogdan Pilyavets', 'email': 'rupilbo@yandex-team.ru', 'color': '#795548', 'profilePicture': None, 'initials': 'BP', 'week_start_day': None, 'global_font_support': False, 'timezone': 'Europe/Moscow'}


In [4]:
# Получим айдишник команды, к которой у нас есть доступ:
team_id = test_class.get_teams(id_only=True)
print(team_id)

4514698


In [5]:
# получим всю инфу про интересующий нас спейс (в данном случае CRM)
space = test_class.get_space_by_name(team_id, name='CRM')
print(space)

{'id': '6849848', 'name': 'CRM', 'private': False, 'statuses': [{'id': 'p6849848_hGuYhhYV', 'status': 'Open', 'type': 'open', 'orderindex': 0, 'color': '#d3d3d3'}, {'id': 'p6849848_aJ5x29At', 'status': 'concept', 'type': 'custom', 'orderindex': 1, 'color': '#FFCC00'}, {'id': 'p6849848_uHca5I69', 'status': 'in progress', 'type': 'custom', 'orderindex': 2, 'color': '#FF540D'}, {'id': 'p6849848_rQtXfgXn', 'status': 'running', 'type': 'custom', 'orderindex': 3, 'color': '#FF00DF'}, {'id': 'p6849848_Yuo3uSOo', 'status': 'review', 'type': 'custom', 'orderindex': 4, 'color': '#A875FF'}, {'id': 'p6849848_e0PVKVnw', 'status': 'Closed', 'type': 'closed', 'orderindex': 5, 'color': '#6bc950'}], 'multiple_assignees': True, 'features': {'due_dates': {'enabled': True, 'start_date': True, 'remap_due_dates': False, 'remap_closed_due_date': False}, 'sprints': {'enabled': True}, 'time_tracking': {'enabled': True, 'harvest': False, 'rollup': False}, 'points': {'enabled': True}, 'custom_items': {'enabled':

In [6]:
# получим всю инфу про интересующие нас листы (в данном случае 'Учащиеся' и 'Лиды (родители)')
studs = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Учащиеся')
pars = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Лиды (родители)')

In [7]:
tags = test_class.get_tags(space['id'])
tags_names = [tag['name'] for tag in tags['tags']]
tags_to_keep = [quote_plus(tag) for tag in tags_names if tag not in ['тестовый', 'технический']]
tags_to_keep.append('')

In [8]:
#заберем данные по таскам 
tasks = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[pars['id']])

page=0&list_ids[]=44610695&include_closed=true
page=1&list_ids[]=44610695&include_closed=true
page=2&list_ids[]=44610695&include_closed=true
page=3&list_ids[]=44610695&include_closed=true
page=4&list_ids[]=44610695&include_closed=true
page=5&list_ids[]=44610695&include_closed=true
page=6&list_ids[]=44610695&include_closed=true
page=7&list_ids[]=44610695&include_closed=true
page=8&list_ids[]=44610695&include_closed=true
page=9&list_ids[]=44610695&include_closed=true
page=10&list_ids[]=44610695&include_closed=true
page=11&list_ids[]=44610695&include_closed=true
page=12&list_ids[]=44610695&include_closed=true
page=13&list_ids[]=44610695&include_closed=true
page=14&list_ids[]=44610695&include_closed=true
page=15&list_ids[]=44610695&include_closed=true
page=16&list_ids[]=44610695&include_closed=true
page=17&list_ids[]=44610695&include_closed=true


In [10]:
#поскольку get_all_tasks возвращает нам список nested джейсонов, напишем функцию, которая сложит все это добро в один датафрейм
#подробнее тут https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
def normalize_wrapper(data:List[dict], **kwargs) -> pd.DataFrame:
    """
    replaces the empty list with a dummy dictionary
    """
    for i in data:
        if not i['tags']:
            i['tags']=[{'name': "fake"}]  #TODO параметризовать ключи
    return pd.json_normalize(data, **kwargs)

def df_getter(data:List[dict], key_getter='tasks', **kwargs) -> pd.DataFrame:
    """
    transforms a list of jsons into a single dataframe using pd.json_normalize
    """
    res = list()
    for chunk in data:
        res.append(normalize_wrapper(chunk[key_getter], **kwargs))
    return pd.concat(res)

In [40]:
par_names = df_getter(tasks,
                      record_path='custom_fields',
                      meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                           ['status', 'status'], ['status', 'type'],
                           ['creator', 'email'], 'due_date', 'start_date', 'tags'],
                      meta_prefix='standard_')
#в последней странице почему-то остается только словарь, сейчас нет времени разбираться
missing_list = par_names.loc[par_names['standard_tags']=={'name': 'fake'}]     
par_names.loc[par_names['standard_tags']=={'name': 'fake'}, 'standard_tags'] = pd.Series([[{'name': 'fake'}]]*len(missing_list))

672 672


,id,name,type,date_created,hide_from_guests,value,required,type_config.default,type_config.placeholder,type_config.new_drop_down,...,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date,standard_tags
667,55255fc1-21d4-45cb-9ddd-428c362de104,Телефон,phone,1615452975390,False,+7 911 238 01 25,False,NaN,NaN,NaN,...,записали на кураторскую встречу; встреча прошл...,1611822911812,1624877654516,None,7. отказ после встречи,custom,esinamarie@yandex-team.ru,1615010400000,None,[{'name': 'fake'}]
668,6477046f-587e-4e74-9063-f9ed0c9592bc,был ли опыт ДО по математике,checkbox,1613735608287,False,NaN,False,NaN,NaN,NaN,...,записали на кураторскую встречу; встреча прошл...,1611822911812,1624877654516,None,7. отказ после встречи,custom,esinamarie@yandex-team.ru,1615010400000,None,[{'name': 'fake'}]
669,0405e9bb-65f7-46e1-ba48-f3fc8fd03827,класс ученика,drop_down,1623139785123,False,NaN,False,0.0,NaN,NaN,...,записали на кураторскую встречу; встреча прошл...,1611822911812,1624877654516,None,7. отказ после встречи,custom,esinamarie@yandex-team.ru,1615010400000,None,[{'name': 'fake'}]
670,a8df8f2b-0eab-4b11-aedc-26f18397c902,комментарий для преподавателя,text,1612451352639,False,Отношения и пропорции в целом не сложная тема ...,False,NaN,NaN,NaN,...,записали на кураторскую встречу; встреча прошл...,1611822911812,1624877654516,None,7. отказ после встречи,custom,esinamarie@yandex-team.ru,1615010400000,None,[{'name': 'fake'}]
671,6397aa72-06d2-4bc9-b4ab-6191f5b58919,причина отказа,drop_down,1615468023005,False,11,False,0.0,NaN,True,...,записали на кураторскую встречу; встреча прошл...,1611822911812,1624877654516,None,7. отказ после встречи,custom,esinamarie@yandex-team.ru,1615010400000,None,[{'name': 'fake'}]


In [41]:
# вытащим теги, чтобы потом выкинуть тестовиков
tags = par_names[['standard_tags', 'standard_id']]
#отберем релевантные столбцы и пивотнем кастомные поля
par_names = par_names[par_names['name'].isin(['Источник трафика', 'Канал привлечения', 'причина отказа', 'Дата заявки'])][['name', 'value', 'standard_name', 'standard_id', 'standard_text_content']]
par_names = par_names.pivot(index=['standard_name', 'standard_id', 'standard_text_content'], columns='name', values='value').reset_index()
par_names = par_names.set_index('standard_id')
#резберем строку с utm-метками и превратим ее в 3 столбца
par_names['Источник трафика'] = par_names['Источник трафика'].fillna("")
par_names['utms'] = par_names['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
par_names['keys'] = par_names['Источник трафика'].str.findall(r"([\w\-_]+)(?==)")
#astype здесь используется как способ конвертнуть пустые и непустые листы в False и True соответственно
mask = par_names['keys'].astype(bool)
antimask = par_names['keys'].astype(bool) == False
utms = par_names.loc[mask, ('utms', 'keys')]
no_utms = par_names.loc[antimask, ('Источник трафика', 'utms')]
#Конструируем словарь utm-ок для тех у кого они заполнены
utms['dict'] = utms.apply(lambda x: {x['keys'][i]:x['utms'][i] for i in range(len(x['keys']))}, axis=1)
#Конструируем словарь utm-ок по умолчанию, для тех у кого они не прокинуты
no_utms['dict'] = no_utms.apply(lambda x: {"utm_source":x['Источник трафика']}, axis=1)
#Собираем это воедино, разбиваем на столбцы и джойним к родительскому датафрейму
full_utms = pd.concat([utms['dict'], no_utms['dict']])
stand_index = full_utms.index
full_utms = pd.DataFrame(full_utms.tolist())
full_utms = full_utms.set_index(stand_index)
par_names = pd.concat([par_names, full_utms], axis=1)

In [42]:
#отдельно обработаем случай с автоматически сгенеренными лидами с fb
par_names.loc[par_names['Источник трафика']=='fb_leads', 'utm_campaign'] = 'fb_leads'
#par_names.loc[par_names['standard_text_content'].str.contains('fb_leads', na=False), ('utm_source', 'utm_campaign')] = ('fb', 'leads')
par_names = par_names.reset_index().rename({'index': 'standard_id'}, axis=1)
par_names = par_names[['standard_name', 'standard_id', 'utm_source', 'utm_campaign', 'utm_term', 'utm_medium', 'utm_content', 'причина отказа', 'Дата заявки']]
par_names['standard_name'] = par_names['standard_name'].replace(r'\n', ' ', regex=True)
#Конвертнем дату заявки в дату
par_names['Дата заявки'] = (par_names['Дата заявки'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")).dt.date

In [43]:
#сформируем список тестовых айдишников
tags_df= tags.explode('standard_tags')
tags_df['name'] = tags_df['standard_tags'].apply(lambda x: x.get('name'))
throwaway = tags_df.query("name == 'тестовый' or name == 'технический' ")['standard_id'].unique()


13


,standard_tags,standard_id
266,[{'name': 'fake'}],k3by2q
267,[{'name': 'fake'}],k3by2q
268,[{'name': 'fake'}],k3by2q
269,[{'name': 'fake'}],k3by2q
270,[{'name': 'fake'}],k3by2q
271,[{'name': 'fake'}],k3by2q
272,[{'name': 'fake'}],k3by2q
273,[{'name': 'fake'}],k3by2q
274,[{'name': 'fake'}],k3by2q
275,[{'name': 'fake'}],k3by2q


In [44]:
#получим плоский лист всех родительских айдишников
parent_ids = np.setdiff1d(par_names['standard_id'].unique(), throwaway)
task_ids = list(parent_ids)
history = test_class.get_time_in_status(task_ids)

1635


In [45]:
#поскольку get_time_in_status возвращает нам список словарей,
#в роли ключей у которых выступают родительские айдишники
#соберем это в единый словарь и превратим в датафрейм
flattened = dict()
for task_collection in history:
    flattened.update(task_collection)
flat_history = {task_id:flattened[task_id]['status_history'] for task_id in flattened}
flat_df = pd.DataFrame.from_dict(flat_history, orient='index')
#сейчас у нас столбцы со словарями для истории каждого статуса - пометим их префиксом
columns = ['col'+str(i) for i in range(len(flat_df.columns))]
flat_df.columns = columns
flat_df = flat_df.reset_index()
#разберем вложенные словари в каждой группе столбцов через json_normalize
#и дропнем столбцы со словарями
flat_df = flat_df.where(flat_df.notna(), lambda x: [{}])
for col in columns:
    flat_df = flat_df.join(pd.json_normalize(flat_df[col]).add_prefix(col+'_'))
flat_df = flat_df.drop(columns=columns)
flat_df.head()


,index,col0_status,col0_color,col0_type,col0_orderindex,col0_total_time.by_minute,col0_total_time.since,col1_status,col1_color,col1_type,...,col13_status,col13_color,col13_type,col13_total_time.by_minute,col13_total_time.since,col14_status,col14_color,col14_type,col14_total_time.by_minute,col14_total_time.since
0,119e7p5,0. новый лид,#d3d3d3,open,0,604,1625603197254,3.слот встречи назначен,#f9d900,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119eteq,0. новый лид,#d3d3d3,open,0,566,1625606086585,2.перезвонить на неделе,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119fjhx,0. новый лид,#d3d3d3,open,0,510,1625609723280,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119g3bp,0. новый лид,#d3d3d3,open,0,425,1625615578207,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,119gcy3,0. новый лид,#d3d3d3,open,0,366,1626273866613,1. недозвон (нет контакта),#800000,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
#превратим нашу таблицу из широкой в длинную и избавимся от префиксов для столбцов с одинановой сутью
#например, для time_by_minute (время в статусе)
melted_df = flat_df.fillna("").melt(id_vars=['index'])
melted_df['variable'] = melted_df['variable'].str.split("_", n=1).str[1]
#выкинем бесполезные столбцы и сделаем родительские айдишники индексами
melted_df = melted_df[~melted_df['variable'].isin(['orderindex', 'color'])]
melted_df = melted_df.set_index('index')
#соберем список отдельных датафреймов для каждого статуса и схлопнем их в одну таблицу
#NB при итерировании по групбаю он возвращает сначала ключ, а потом датафрейм по этому ключу
to_concat = [y for x,y in melted_df.groupby('variable', as_index=False)]
new_df = pd.concat(to_concat, axis=1)
#дропнем нерелевантные столбцы и переименуем оставшиеся
col_names = list(new_df.iloc[0, [i for i in range(0,len(new_df.columns),2)]])
new_df = new_df.iloc[:,[i for i in range(1,len(new_df.columns),2)]]
new_df.columns = col_names
new_df = new_df.rename(columns={'total_time.by_minute':'min_passed', 'total_time.since': 'start_dt'})
#конвертнем столбец со временем в формат даты
new_df['start_dt'] = new_df['start_dt'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")
new_df.head()

,status,min_passed,start_dt,type
index,,,,
119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open
119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open
119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open
119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open
119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open


In [47]:
#получим список всех детей из кликапа
kids = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[studs['id']])

page=0&list_ids[]=46598432&include_closed=true
page=1&list_ids[]=46598432&include_closed=true
page=2&list_ids[]=46598432&include_closed=true


In [48]:
#как и родителями, сложим все в один датафрейм
df = df_getter(kids,
               record_path='custom_fields',
               meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                    ['status', 'status'], ['status', 'type'],
                    ['creator', 'email'], 'due_date', 'start_date'],
               meta_prefix="standard_")
df.columns

Index(['id', 'name', 'type', 'date_created', 'hide_from_guests', 'required',
       'value', 'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id',
       'type_config.options', 'standard_id', 'standard_name',
       'standard_text_content', 'standard_date_created',
       'standard_date_updated', 'standard_date_closed',
       'standard_status.status', 'standard_status.type',
       'standard_creator.email', 'standard_due_date', 'standard_start_date'],
      dtype='object')

In [49]:
#выкинем тестовых юзеров
df = df[df['standard_status.status']!='тестовый']
df.sample(5)

,id,name,type,date_created,hide_from_guests,required,value,type_config.fields,type_config.field_inverted_name,type_config.linked_subcategory_access,...,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
405,35dec42d-8229-417d-871a-1abf97209c77,класс,short_text,1622121639374,False,False,5 (учится по программе 6),NaN,NaN,NaN,...,Фёдор Михайлович Руденко,\n,1622194984810,1628524629527,None,платящий (2+ покупок),custom,k-perfileva@yandex-team.ru,1627520400000,None
696,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,Анна Ионова,NaN,NaN,NaN,...,Евгений Богданов,\n,1619073528842,1627979246702,None,"понравилось, думает",custom,k-perfileva@yandex-team.ru,1628902800000,None
858,b82a5972-5fb8-4be4-bcef-af0fbc272c05,Shared with me,list_relationship,1620249016705,False,False,NaN,[],NaN,False,...,Настя Котляревская,None,1614191854722,1619530067019,1619530051903,отвал (скрыть ученика),closed,irawhitelake@gmail.com,None,None
505,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,NaN,NaN,NaN,...,Амира Ланова,\n,1627221642562,1627222132459,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
1071,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': 'zngzg9', 'name': 'Наталья Рямбова', '...",[{'field': 'cf_b261ecb0-58b9-4a4e-bcae-1e1cafb...,Учащийся,True,...,Богдан Рямбов,\n,1625748777806,1625749094931,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None


In [50]:
#удалим не нужные нам столбцы
to_drop = ['type_config.default', 'type_config.placeholder', 'type_config.new_drop_down', 'type_config.options',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id']
df = df.drop(columns=df.columns.intersection(to_drop))
df.head()

,id,name,type,date_created,hide_from_guests,required,value,standard_id,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
0,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,u3runn,Олег Диганов,\n,1628772207379,1628772239026,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
1,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,u3runn,Олег Диганов,\n,1628772207379,1628772239026,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
2,4a4363cf-e27e-4b59-b781-63cb161a43e3,Расписание,short_text,1613142022298,False,False,NaN,u3runn,Олег Диганов,\n,1628772207379,1628772239026,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
3,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': 'j33grw', 'name': 'Анна Диганова', 'st...",u3runn,Олег Диганов,\n,1628772207379,1628772239026,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
4,e6cb4ac4-e138-4895-968d-9ebccaa12c78,Согласие на видеозапись,labels,1613132651997,False,False,[8472ccaa-0c6e-4537-b71c-f2345ec26b11],u3runn,Олег Диганов,\n,1628772207379,1628772239026,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None


In [51]:
#пивотнем столбец name (каждое уникальное значение из этого столбца становится отдельным столбцом в пивотизированном датафрейме)
#поскольку столбец 'родитель в воронке лидов' в кликапе является ссылкой на другой лист, API возвращает нам этот лист, а пандас
#читает его как питоновский лист. В нашем случае в нем всегда один элемент, поэтому explode не увеличивает кол-во строк в датафрейме
#а просто избавляет нас от листа в cтолбце, превращая ее в столбец словарей
df2 = df.copy()
df2 = df2.pivot(index='standard_id', columns='name', values='value')
df2 = df2.explode('Родитель в воронке лидов')
df2.head()

name,Shared with me,Инциденты,Преподаватель списка учащиеся,Прогулы,Пройденные занятия пакета (старое),Расписание,Родитель в воронке лидов,Согласие на видеозапись,Цель ученика и сроки достижения цели,класс,купленные пакеты,первое занятие
standard_id,,,,,,,,,,,,
6fynn4,[],NaN,Дарья Карпухина,NaN,11\02 перерыв с 22\02 до 28\02,"чт 18-00 - 60 мин, пн 20-00 - 15 мин","{'id': '69yhrv', 'name': 'Татьяна Рушечникова ...",[1171db5a-754c-4178-85f1-67847ab15f03],"Дочь 6 класс, не очень любит математику, после...",NaN,"[{'id': 'ejzw6t', 'name': 'Варя Рушечникова па...",1613005200000
6fynn5,NaN,NaN,"Светлана Зайцева (15 мин), Сергей Христолюбов ...",NaN,16\02,"вт 16-00 60 мин, чт 16-00 15 мин","{'id': '6fu8tt', 'name': 'Анна Гуляева', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6w', 'name': 'Иван Гуляев пакет 4...",1613437200000
6fyppm,NaN,NaN,"Светлана Зайцева (15 мин), Анна И (60 мин)",NaN,NaN,"вскр 14-00 - 60 мин, вт 16-00 - 15 мин","{'id': 'd6zdz6', 'name': 'Ольга', 'status': '9...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'fk0rm6', 'name': 'Даша пакет 4+4 купл...",1612054800000
6hqz5e,[],NaN,Антон Загривин,"[{'id': 'rp4ytc', 'name': 'Аня Таран', 'status...",15\02,"пн 12-00 60 мин, чт 12-00 15 мин","{'id': '6dy7aj', 'name': 'Марина Таран', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6r', 'name': 'Аня пакет 4+4 купле...",1613350800000
6huakn,NaN,NaN,Света и Никита,NaN,27\02,"среда 17-00 60 мин, пятница 18-30 15 мин","{'id': '6fut3z', 'name': 'Наталья Кузнецова', ...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6c', 'name': 'Вася Кузнецов пакет...",1613523600000


In [52]:
#читаем наш столбец со словарями в отдельный датафрейм, добавляем префикс к именам столбцов
#чтобы избежать конфликтов при последующем мердже
parent = df2['Родитель в воронке лидов'].apply(pd.Series).add_prefix("par_")
parent = parent[['par_id', 'par_name']]
parent.head()

,par_id,par_name
standard_id,,
6fynn4,69yhrv,Татьяна Рушечникова (Болгова)
6fynn5,6fu8tt,Анна Гуляева
6fyppm,d6zdz6,Ольга
6hqz5e,6dy7aj,Марина Таран
6huakn,6fut3z,Наталья Кузнецова


In [53]:
#проделываем то же самое для столбца со ссылкой на купленные пакеты
packages = df2['купленные пакеты'].explode().apply(pd.Series).add_prefix("purch_")
packages = packages[['purch_id']]
packages.head()

,purch_id
standard_id,
6fynn4,ejzw6t
6fynn4,71t1d4
6fynn4,7dyck9
6fynn4,gz8w39
6fynn4,gk7nub


In [54]:
#получаем инфу о списке покупок в кликапе
purch_list = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Заявки на покупку')
print(purch_list)

{'id': '48853634', 'name': 'Заявки на покупку', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 285, 'due_date': None, 'start_date': None, 'folder': {'id': '23627065', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': True, 'permission_level': 'create'}


In [55]:
#забираем все таски из этого списка
purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list['id']])

page=0&list_ids[]=48853634&include_closed=true
page=1&list_ids[]=48853634&include_closed=true
page=2&list_ids[]=48853634&include_closed=true
page=3&list_ids[]=48853634&include_closed=true
page=4&list_ids[]=48853634&include_closed=true


In [56]:
#и внвь соберем эту инфу в плоский датафрейм, выкинем тесовых и оставим только релевантные столбцы
purch_df = df_getter(purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
purch_df = purch_df[(purch_df['name']=='Ссылка на ребенка в админке') & (purch_df['standard_status.status']!='тестовый')][['value', 'standard_id']].dropna()
#вытащим praktikum_id из юрла
purch_df['praktikum_id'] = purch_df['value'].str.split("/").str[-1]
purch_df.head()

,value,standard_id,praktikum_id
3,https://praktikum-admin.yandex-team.ru/math/st...,u3rtvj,12046862
13,https://praktikum-admin.yandex-team.ru/math/st...,u3qqz9,10937593
23,https://praktikum-admin.yandex-team.ru/math/st...,u3pr2w,12042389
33,https://praktikum-admin.yandex-team.ru/math/st...,t1jtx0,7722621
43,https://praktikum-admin.yandex-team.ru/math/st...,t1eenh,12024203


In [57]:
#сопоставим айдишники детей с их айдишниками в практикуме
kids_links = packages.reset_index().merge(purch_df, how='left', left_on='purch_id', right_on='standard_id')
kids_links = kids_links[['purch_id', 'praktikum_id', 'standard_id_x']].groupby('standard_id_x')['praktikum_id'].last()
kids_links.head()

standard_id_x
6fynn4    7210563
6fynn5    7203187
6fyppm       None
6hqz5e    7245837
6huakn       None
Name: praktikum_id, dtype: object

In [58]:
#теперь сджойним детей с их родителями
full_lineage = parent.merge(kids_links, how='left', left_index=True, right_index=True)
full_lineage.sample(5)

,par_id,par_name,praktikum_id
standard_id,,,
nvg4xe,131nz8m,Елена Чернухина,11054266
nh7w3p,n98k4f,Леонид Моторин,10781411
nxajzf,119h0p7,Галина Пеншина,11102355
ae48z6,p984ar,Наталия,11432475
q19q68,pvfjt8,Мария Исаева,11753936


In [59]:
#наконец, смерджим все это воедино с датафреймом родительских атрибутов
fin_df = new_df.reset_index().merge(full_lineage, how='left', left_on='index', right_on='par_id')
fin_df = fin_df[fin_df['type']!=""]
fin_df = fin_df.merge(par_names, how='left', left_on='index', right_on='standard_id')
fin_df.sample(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_term,utm_medium,utm_content,причина отказа,Дата заявки
1738,139kyj3,1. недозвон 1 сутки,389,2021-07-12 09:29:10.022,custom,NaN,NaN,NaN,Настя Сальникова,139kyj3,fb_leads,fb_leads,NaN,NaN,NaN,0,2021-07-11
7094,d6zedj,отказ или отвал,146612.0,2021-03-11 16:11:46.276,closed,NaN,NaN,NaN,Дарья,d6zedj,google_ads,NaN,NaN,NaN,NaN,0,2021-01-27
2995,x99ep4,1. недозвон 1 сутки,2542,2021-06-10 16:48:31.556,custom,NaN,NaN,NaN,Светлана Кошкина (FB leads via Zapier),x99ep4,fb_leads,fb_leads,NaN,NaN,NaN,0,2021-06-07
5418,pza5dp,1. недозвон (пингуем нед.),10069.0,2021-08-01 14:49:13.602,custom,NaN,NaN,NaN,Ажара,pza5dp,facebook,math_fb_traffic_september_offer,NaN,cpc,offer3_yellow_sczapishis_v_september,0,2021-07-29
750,hd5y0a,"2. хочет, но в будущем",75064,2021-06-21 13:14:29.225,custom,NaN,NaN,NaN,Анна Кожемякина,hd5y0a,facebook_lead,NaN,NaN,NaN,NaN,NaN,2021-04-18
3201,zkmya1,2.перезвонить на неделе,5805,2021-07-03 16:21:58.670,custom,NaN,NaN,NaN,Гузель Чебаненко,zkmya1,,NaN,NaN,NaN,NaN,11,2021-07-03
6290,pfbqzm,1. недозвон (нет контакта),13233.0,2021-08-03 11:45:37.677,custom,NaN,NaN,NaN,Александра,pfbqzm,,NaN,NaN,NaN,NaN,0,2021-07-22
3766,1begyc0,1. недозвон 2 сутки,411.0,2021-08-07 11:29:58.632,custom,NaN,NaN,NaN,Альбина Константинова,1begyc0,,NaN,NaN,NaN,NaN,NaN,2021-08-06
4837,15zxu3b,2. отказ после контакта,28940.0,2021-07-23 13:57:41.279,custom,NaN,NaN,NaN,Лейсан Мингазова,15zxu3b,,NaN,NaN,NaN,NaN,15,2021-07-19
6224,khc3u4,отказ или отвал,29092.0,2021-05-27 14:25:54.871,closed,NaN,NaN,NaN,Наталья,khc3u4,fb_leads,fb_leads,NaN,NaN,NaN,11,2021-05-27


In [60]:
#добавим мэппинг статусов из файла
statuses = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='statuses', index_col=1)
fin_df = fin_df.merge(statuses, left_on='status', right_index=True, how='left')
fin_df.head(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,...,utm_campaign,utm_term,utm_medium,utm_content,причина отказа,Дата заявки,этап,Созвон,Бронирование,Встреча
0,119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open,119e7p5,Светлана (Иван),10994925,Светлана (Иван),119e7p5,...,NaN,NaN,NaN,NaN,NaN,2021-07-06,0 заявка,0.0,0.0,0.0
1,119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open,NaN,NaN,NaN,Александра Борисова,119eteq,...,fb_leads,NaN,NaN,NaN,11,2021-07-07,0 заявка,0.0,0.0,0.0
2,119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open,NaN,NaN,NaN,КАТЕРИНА,119fjhx,...,fb_leads,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
3,119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open,119g3bp,Анна(Денис),11200929,Анна(Денис),119g3bp,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
4,119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open,NaN,NaN,NaN,Марина Белькова,119gcy3,...,fb_leads,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
5,119gmeg,0. новый лид,303,2021-07-07 05:00:28.312,open,NaN,NaN,NaN,Светлана Керебко,119gmeg,...,NaN,NaN,NaN,NaN,15,2021-07-07,0 заявка,0.0,0.0,0.0
6,119gr05,0. новый лид,277,2021-07-07 05:28:11.314,open,NaN,NaN,NaN,Светлана Реутова,119gr05,...,NaN,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
7,119h0p7,0. новый лид,174,2021-07-07 07:20:50.794,open,119h0p7,Галина Пеншина,11102355,Галина Пеншина,119h0p7,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
8,119h459,0. новый лид,139,2021-07-07 07:52:26.718,open,NaN,NaN,NaN,светлана иванова 888,119h459,...,fb_leads,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
9,119h57w,0. новый лид,128,2021-07-20 19:34:56.653,open,NaN,NaN,NaN,Светлана Андреевская,119h57w,...,fb_leads,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0


In [61]:
#создадим отдельные столбцы с датой созвона, брони и встречи
fin_df['call'] = fin_df['start_dt'].where(fin_df['Созвон']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['meeting'] = fin_df['start_dt'].where(fin_df['Встреча']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['booking'] = fin_df['start_dt'].where(fin_df['Бронирование']==1, np.NaN).groupby(fin_df['index']).transform('min')
#выкинем лишние столбцы
fin_df = fin_df.drop(columns=['Созвон', 'Встреча', 'par_name', 'index', 'par_id']).rename(columns={'причина отказа': 'churn_reason'})
fin_df['lead_date'] = fin_df.groupby('standard_id')['start_dt'].transform('min')
fin_df['lead_date'] = fin_df['Дата заявки'].combine_first(fin_df['lead_date'])
#конвертнем дату заявки в datetime, предварительно проверив на пустоту
if len(fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date']) > 0:
    fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date'] = fin_df[fin_df['Дата заявки'].isna()]['lead_date'].apply(pd.to_datetime).dt.date
fin_df = fin_df.drop(columns=['Дата заявки', 'Бронирование'])
fin_df = fin_df.rename(columns={'этап': 'etap'})
fin_df.sample(7)

,status,min_passed,start_dt,type,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_term,utm_medium,utm_content,churn_reason,etap,call,meeting,booking,lead_date
2025,1. недозвон 1 сутки,2567,2021-07-25 18:28:31.006,custom,NaN,Ксения Попова,18bvq70,,NaN,NaN,NaN,NaN,0,1 устанавливаем контакт,2021-07-29 12:47:42.787,NaT,NaT,2021-07-25
5773,1. недозвон (нет контакта),36101.0,2021-07-18 14:36:55.013,custom,NaN,Анна Просянникова,11bmpdv,,NaN,NaN,NaN,NaN,0,1 устанавливаем контакт,NaT,NaT,NaT,2021-07-08
373,0. новый лид,25,2021-07-24 18:34:30.402,open,11588842,Дарима Лодойбалова,18btk7t,,NaN,NaN,NaN,NaN,NaN,0 заявка,2021-07-24 19:00:31.944,2021-07-25 12:20:08.908,2021-07-24 19:00:31.944,2021-07-24
2462,1. недозвон 1 сутки,299,2021-05-13 11:11:14.750,custom,8569142,Татьяна Архипова,jt48ry,yandex,Yandex_Search_Kids_Math,курсы по математике,NaN,NaN,NaN,1 устанавливаем контакт,2021-05-13 16:10:22.352,2021-05-23 15:09:34.608,2021-05-13 16:10:22.352,2021-05-12
5017,5.встреча подтверждена,107.0,2021-08-11 17:04:05.796,custom,NaN,Елена (Таисия),1axb85t,,NaN,NaN,NaN,NaN,NaN,4 подтверждаем встречу,2021-08-03 18:48:47.735,2021-08-11 18:51:52.105,2021-08-03 18:48:47.735,2021-08-03
2137,1. недозвон 1 сутки,458,2021-07-31 09:17:08.111,custom,NaN,Дарья Васильева,1ag1p9e,fb_leads,fb_leads,NaN,NaN,NaN,15,1 устанавливаем контакт,2021-07-31 16:55:48.249,NaT,NaT,2021-07-30
6752,отказ после контакта,6850.0,2021-06-16 19:03:41.639,custom,NaN,Юлия Гришина,h79980,facebook_lead,NaN,NaN,NaN,NaN,11,na,2021-04-16 10:21:08.841,NaT,NaT,2021-04-15


In [62]:
fin_df['standard_id'].nunique()

1635

In [63]:
fin_df.to_csv('fin1_df.csv')

# ЗЭ ЭНД